This notebook is part of the $\omega radlib$ documentation: https://docs.wradlib.org.

Copyright (c) $\omega radlib$ developers.
Distributed under the MIT License. See LICENSE.txt for more info.

# xarray CfRadial1 backend

In this example, we read and write CfRadial1 data files using the xarray `cfradial1` backend.

In [ ]:
import wradlib as wrl
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as pl
import numpy as np
import xarray as xr
try:
    get_ipython().magic("matplotlib inline")
except:
    pl.ion()

## Load CfRadial1 Volume Data

In [ ]:
fpath = 'netcdf/cfrad.20080604_002217_000_SPOL_v36_SUR.nc'
f = wrl.util.get_wradlib_data_file(fpath)
vol = wrl.io.open_cfradial1_dataset(f)

### Fix several issues of CfRadial1 azimuth's

In [ ]:
for i, swp in enumerate(vol):
    num_rays = int(360 // swp.azimuth.diff("azimuth").median())
    start_rays = swp.dims["azimuth"] - num_rays
    vol[i] = swp.isel(azimuth=slice(start_rays, start_rays + num_rays )).sortby("azimuth")

### Inspect RadarVolume

In [ ]:
display(vol)

### Inspect root group

The `sweep` dimension contains the number of scans in this radar volume. Further the dataset consists of variables (location coordinates, time_coverage) and attributes (Conventions, metadata).

In [ ]:
vol.root

### Inspect sweep group(s)

The sweep-groups can be accessed via their respective keys. The dimensions consist of `range` and `time` with added coordinates `azimuth`, `elevation`, `range` and `time`. There will be variables like radar moments (DBZH etc.) and sweep-dependend metadata (like `fixed_angle`, `sweep_mode` etc.).

In [ ]:
display(vol[0])

### Goereferencing

In [ ]:
swp = vol[0].copy().pipe(wrl.georef.georeference_dataset)

### Plotting

In [ ]:
swp.DBZ.plot.pcolormesh(x='x', y='y')
pl.gca().set_aspect('equal')

In [ ]:
fig = pl.figure(figsize=(10,10))
swp.DBZ.wradlib.plot_ppi(proj='cg', fig=fig)

In [ ]:
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

map_trans = ccrs.AzimuthalEquidistant(central_latitude=swp.latitude.values, 
                                      central_longitude=swp.longitude.values)

In [ ]:
map_proj = ccrs.AzimuthalEquidistant(central_latitude=swp.latitude.values, 
                                      central_longitude=swp.longitude.values)
pm = swp.DBZ.wradlib.plot_ppi(proj=map_proj)
ax = pl.gca()
ax.gridlines(crs=map_proj)
print(ax)

In [ ]:
map_proj = ccrs.Mercator(central_longitude=swp.longitude.values)
fig = pl.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection=map_proj)
pm = swp.DBZ.wradlib.plot_ppi(ax=ax)
ax.gridlines(draw_labels=True)

In [ ]:
import cartopy.feature as cfeature
def plot_borders(ax):
    borders = cfeature.NaturalEarthFeature(category='physical',
                                           name='coastline',
                                           scale='10m',
                                           facecolor='none')
    ax.add_feature(borders, edgecolor='black', lw=2, zorder=4)

map_proj = ccrs.Mercator(central_longitude=swp.longitude.values)
fig = pl.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection=map_proj)

DBZ = swp.DBZ
pm = DBZ.where(DBZ > 0).wradlib.plot_ppi(ax=ax)
plot_borders(ax)
ax.gridlines(draw_labels=True)

In [ ]:
import matplotlib.path as mpath
theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)

map_proj = ccrs.AzimuthalEquidistant(central_latitude=swp.latitude.values, 
                                     central_longitude=swp.longitude.values,
                                    )
fig = pl.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection=map_proj)
ax.set_boundary(circle, transform=ax.transAxes)

pm = swp.DBZ.wradlib.plot_ppi(proj=map_proj, ax=ax)
ax = pl.gca()
ax.gridlines(crs=map_proj)

In [ ]:
fig = pl.figure(figsize=(10, 8))
proj=ccrs.AzimuthalEquidistant(central_latitude=swp.latitude.values, 
                               central_longitude=swp.longitude.values)
ax = fig.add_subplot(111, projection=proj)
pm = swp.DBZ.wradlib.plot_ppi(ax=ax)
ax.gridlines()

In [ ]:
swp.DBZ.wradlib.plot_ppi()

### Inspect radar moments

The DataArrays can be accessed by key or by attribute. Each DataArray has dimensions and coordinates of it's parent dataset. There are attributes connected which are defined by Cf/Radial standard.

In [ ]:
display(swp.DBZ)

### Create simple plot

Using xarray features a simple plot can be created like this. Note the `sortby('time')` method, which sorts the radials by time.

In [ ]:
swp.DBZ.sortby('rtime').plot(x="range", y="rtime", add_labels=False)

In [ ]:
fig = pl.figure(figsize=(5,5))
pm = swp.DBZ.wradlib.plot_ppi(proj={'latmin': 33e3}, fig=fig)

### Mask some values

In [ ]:
swp['DBZ'] = swp['DBZ'].where(swp['DBZ'] >= 0)
swp['DBZ'].plot()

### Export to ODIM and CfRadial2

In [ ]:
vol.to_odim('cfradial1_as_odim.h5')
vol.to_cfradial2('cfradial1_as_cfradial2.nc')

### Import again

In [ ]:
vola = wrl.io.open_odim_dataset('cfradial1_as_odim.h5', decode_coords=True, 
                                backend_kwargs=dict(keep_azimuth=True,
                                                   keep_elevation=False))

In [ ]:
vola.root

In [ ]:
volb = wrl.io.open_cfradial2_dataset('cfradial1_as_cfradial2.nc')

### Check equality

Some variables need to be dropped, since they are not exported to the other standards.

In [ ]:
drop = set(vol[0]) ^ set(vola[0]) | set({"elevation", "rtime"})
xr.testing.assert_allclose(vol.root, vola.root)
xr.testing.assert_allclose(vol[0].drop_vars(drop), vola[0].drop_vars(drop, errors="ignore"))
xr.testing.assert_allclose(vol.root, volb.root)
xr.testing.assert_equal(vol[0], volb[0])
xr.testing.assert_allclose(vola.root, volb.root)
xr.testing.assert_allclose(vola[0].drop_vars(drop, errors="ignore"), volb[0].drop_vars(drop, errors="ignore"))

## More CfRadial1 loading mechanisms

### Use `xr.open_dataset` to retrieve explicit group


In [ ]:
swp = xr.open_dataset(f, engine="cfradial1", group="sweep_9")
display(swp)